In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
normal = tfd.Normal(loc=tf.Variable(0., name='loc'), scale=1.)

In [3]:
normal.trainable_variables

(<tf.Variable 'loc:0' shape=() dtype=float32, numpy=0.0>,)

The mean value of this normal distribution is now trainable and can be updated accordingly to some learning principle. The learning principle that we often use when training DL models is maximum likelihood. This is the goal of finding the parameters of our model that maximize the likelihood or probability of our data. Finding the parameters that maximize the likelihood is the same as finding the parameters that minimize the negative log likelihood and in practice this is what is often used.

In [4]:
def nll(x_train):
    return -tf.reduce_mean(normal.log_prob(x_train))

Let's define a custom training loop to learn the mean parameters from the data.

In [5]:
@tf.function
def get_loss_and_grads(x_train):
    with tf.GradientTape() as tape:
        tape.watch(normal.trainable_variables)
        loss=nll(x_train)
    grads = tape.gradient(loss, normal.trainable_variables)
    return loss, grads

In [16]:
x_samples = tfd.Normal(10, 1).sample(100)

In [19]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05)

num_steps = 10
for _ in range(num_steps):
    loss, grads = get_loss_and_grads(x_samples)
    optimizer.apply_gradients(zip(grads, normal.trainable_variables))